# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import os
import shutil
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession,Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Read in the data here
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_spark =spark.read.load('./sas_data')
shutil.rmtree("output", ignore_errors=True, onerror=None)
shutil.rmtree("data", ignore_errors=True, onerror=None)
df_spark.write.parquet("./output/")
df_spark =spark.read.load('./output')

df_spark.write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .option("delimiter","|")\
  .option("mode","overwrite")\
  .save("data/")


In [5]:
df= pd.read_csv("./data/part-00000-2ef4b4c9-2b18-4681-86ab-5ecf2457aafc-c000.csv",sep="|")
df.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,21,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,...,NaN,M,1976.0,10292016,F,NaN,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,...,NaN,M,1984.0,10292016,F,NaN,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,...,NaN,M,1987.0,10292016,M,NaN,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,NaN,M,1987.0,10292016,F,NaN,DL,9.495645e+10,00040,B1
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,NaN,M,1988.0,10292016,M,NaN,DL,9.495639e+10,00040,B1


In [6]:
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [7]:
df_spark.select(col('airline'),col('i94port'),col('i94addr')).distinct()\
.sort(df_spark.airline.desc()).show(truncate=False)


+-------+-------+-------+
|airline|i94port|i94addr|
+-------+-------+-------+
|ZZ     |MIA    |NC     |
|ZZ     |WAS    |NY     |
|ZZ     |SEA    |WA     |
|ZZ     |SFR    |DE     |
|ZZ     |NYC    |NJ     |
|ZZ     |ADW    |null   |
|ZZ     |SFR    |NY     |
|ZZ     |MIA    |FL     |
|ZZ     |BRO    |TX     |
|ZZ     |NYC    |CT     |
|ZZ     |ATL    |NY     |
|ZZ     |NEW    |HI     |
|ZZ     |ADW    |MD     |
|ZZ     |NYC    |NY     |
|ZZ     |HHW    |HI     |
|ZX     |WAS    |IN     |
|ZX     |FTL    |MN     |
|ZX     |TOR    |GA     |
|ZX     |FTL    |OH     |
|ZX     |HHW    |CT     |
+-------+-------+-------+
only showing top 20 rows



In [8]:
df_spark.count()

3096313

In [9]:
df_spark.select([count(when(isnan(c) | col(c).isNull(), c))\
                .alias(c) for c in df_spark.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here
def df_spark_clean(readFormat, input_data, output_data):
    """
    load df_sas data and clean it and write to output_data
    Params:
        readFormat  : format of the input data
        input_data   : stored location of df_sas
        output_data  : location of output data to be saved. 
    """
    # load i94immigration data
    df_sas =spark.read.format(read_format) \
            .load(input_data)
    
    # transfer dates to spark datetype, format column names and datatypes, drop columns and duplicated rows
    df_sas = df_sas.withColumn("data_base_sas", to_date(lit("01/01/1960"), "MM/dd/yyyy")) \
            .withColumn("arrival_date", expr("date_add(data_base_sas, arrdate)")) \
            .withColumn("departure_date", expr("date_add(data_base_sas, depdate)")) \
            .drop("data_base_sas", "arrdate", "depdate") \
            .withColumn("cic_id",col("cicid").cast(IntegerType())).drop("cicid") \
            .withColumn("arrive_year",col('i94yr').cast(IntegerType())).drop("i94yr") \
            .withColumn("arrive_month",col('i94mon').cast(IntegerType())).drop("i94mon") \
            .withColumn("citizen_country",col('i94cit').cast(IntegerType())).drop("i94cit") \
            .withColumn("resident_country",col('i94res').cast(IntegerType())).drop("i94res") \
            .withColumn("age",col('i94bir').cast(IntegerType())).drop("i94bir") \
            .withColumn("birth_year",col('biryear').cast(IntegerType())).drop("biryear") \
            .withColumn("visa_class",col('i94visa').cast(IntegerType())).drop("i94visa") \
            .withColumn("mode",col('i94mode').cast(IntegerType())).drop("i94mode") \
            .withColumn("allowed_date", to_date("dtaddto", "MMddyyyy")) \
            .withColumnRenamed("i94port", "port") \
            .withColumnRenamed("i94addr","arrive_state") \
            .withColumnRenamed("visapost","visa_issue_state") \
            .withColumnRenamed("entdepa","arrive_flag") \
            .withColumnRenamed("entdepd","departure_flag") \
            .withColumnRenamed("matflag","match_flag") \
            .withColumnRenamed("entdepu","update_flag") \
            .withColumnRenamed("fltno","flight_num") \
            .withColumnRenamed("visatype","visa_type") \
            .withColumnRenamed("visapost","visa_issue_state") \
            .withColumnRenamed("occup","occupation") \
            .drop('count','dtadfile','insnum','admnum','dtaddto') \
            .distinct()
    
    # write the end data to parquet file in workspace
    # remove directory if already exist
    if os.path.exists(output_data + 'immigration'):
        os.rmdir(output_data + 'immigration')
    else:
        df_sas.write.partitionBy('arrive_state').parquet('immigration')    


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.